# Уровень потребительской лояльности (NPS -  Net Promoter Score)

**ОПИСАНИЕ ПРОЕКТА**

**Источник данных**: телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.\
**Задача**:
- Определить текущий уровень потребительской лояльности, или NPS (Net Promoter Score), среди клиентов из России на основании проведённого опроса.
- Выявить наиболее и наименее лояльные группы пользователей.
- Охарактеризовать клиентов, которые относятся к группе cторонников (promoters).

>*Для работы использована часть базы данных предварительно выгруженая в SQLite*. 

**Формат представления результатов - дашборды tableau**

>- Дашборды должны упроcтить менеджерам работу с данными. Удобство и вариативность получения информации стали главными ориентирами в работе над проектом.
>- Дашборды сделаны независимыми - настройки одного дашборда не влияют на другой, несмотря на наличие функционально одинаковых элементов.


**<p style="color: #0000FF;">[Ссылка на дашборды (tableau)](https://public.tableau.com/views/NetPromoterScoreforTelecom/NPSstudy?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)</p>**

Ниже в блокноте:
- формирование датасета для tableau (SQL),
- переименование сегментов клиентов в некоторых категориальных полях в более удобные для презентации (Python), 
- проверка итогового csv датасета (Python),
- очистка данных. Выявлен дефект - отрицательные значения днеё "жизни клиента" в 13 строках. Из-за незначительности количества дефектных строк, они были удалены на уровне SQL-запроса.

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# Формируем датасет для построения дашботда

# Создаём поле с информацией о том, является ли клиент новым (количество дней «жизни» не более 365)
# Категориальные поля преобразуем в текстовый вид.
# Результаты опроса, оценки делим на три группы.
# Присоединяем к полям из таблицы user необходимые поля из других таблиц.

query = """
SELECT user_id,
       lt_day,
       CASE
           WHEN lt_day <= 365 THEN 'новые'
           WHEN lt_day > 365 THEN 'старые'
       END AS is_new,
       age,
       
       CASE
           WHEN gender_segment = 0 THEN 'мужчины'
           WHEN gender_segment = 1 THEN 'женщины'
           ELSE 'нет данных'
       END AS gender_segment,
       
       os_name,
       cpe_type_name,
       location.country,
       location.city,
       CAST(age_segment.title AS char(10)) AS age_segment,
       CAST(traffic_segment.title AS char(10)) AS traffic_segment,
       CAST(lifetime_segment.title AS char(10)) AS lifetime_segment,
       nps_score,
       CASE
           WHEN nps_score <= 6 THEN 'критики'
           WHEN nps_score >= 7 AND nps_score <= 8 THEN 'нейтралы'
           WHEN nps_score >= 9 THEN 'сторонники'
       END AS nps_group
       
FROM user
LEFT JOIN location         ON user.location_id = location.location_id
LEFT JOIN age_segment      ON user.age_gr_id   = age_segment.age_gr_id
LEFT JOIN traffic_segment  ON user.tr_gr_id    = traffic_segment.tr_gr_id
LEFT JOIN lifetime_segment ON user.lt_gr_id    = lifetime_segment.lt_gr_id

WHERE user.lt_day >= 0
;
"""

In [4]:
df = pd.read_sql(query, engine)
#df.head(3)

<div class="alert alert-success" style="background-color:#90EE90;">
</div>

In [5]:
# Контролируем объём таблицы на каждома этапе добавления кода SQL
df.shape

(502480, 14)

In [6]:
# Преобразуем названия сегментов для целей презентации.

# Разбиваем строку по пробелу, затем добавляем только элементы от 1 (с пробелом), т.е создаём новый список.
def ext_segment(row):   
    element = row.split(' ')
    return ' '.join(element[1:])

# Создаём для удобства новую переменну. и применяем функцию к интересующим столбцам
df_ext = df
df_ext['age_segment'] = df_ext['age_segment'].apply(ext_segment)
df_ext['traffic_segment'] = df_ext['traffic_segment'].apply(ext_segment)
df_ext['lifetime_segment'] = df_ext['lifetime_segment'].apply(ext_segment)

#df_ext.sample(5)

In [7]:
# Контроль после корректировки данных

#df_ext['age_segment'].value_counts()
#df_ext['traffic_segment'].value_counts()
#df_ext['lifetime_segment'].value_counts()
#df_ext['is_new'].value_counts()

df_defect = df_ext[df_ext['lt_day'] < 0]
df_defect.shape

(0, 14)

In [8]:
# Сохраняем переменную в файл csv
df_ext.to_csv('telecom_tableau.csv', index=False)

In [9]:
# Проверяем сохранённый для tableau файл csv

df2 = pd.read_csv ('telecom_tableau.csv')
#display (df2.sample(5))
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502480 entries, 0 to 502479
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502480 non-null  object 
 1   lt_day            502480 non-null  int64  
 2   is_new            502480 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502480 non-null  object 
 5   os_name           502480 non-null  object 
 6   cpe_type_name     502480 non-null  object 
 7   country           502480 non-null  object 
 8   city              502480 non-null  object 
 9   age_segment       501939 non-null  object 
 10  traffic_segment   502480 non-null  object 
 11  lifetime_segment  502480 non-null  object 
 12  nps_score         502480 non-null  int64  
 13  nps_group         502480 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB
